<a href="https://colab.research.google.com/github/chw8207/Transformer/blob/main/%ED%86%A0%ED%81%AC%EB%82%98%EC%9D%B4%EC%A0%80%26%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B%EB%A7%A4%EC%B9%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece==0.1.94

In [1]:
!pip install gensim
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [27]:
import tensorflow as tf
import math
import torch
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')
%matplotlib inline

### GPU 설정

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0' :
    raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

Found GPU at: /device:GPU:0


In [4]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2051661879056536998
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 40129593344
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6516385104724671060
physical_device_desc: "device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0"
xla_global_id: 416903419
]


### Word2Vec 토큰화

In [14]:
# 데이터 토큰화하고 word2vec 모델을 훈련시키기
sample = open('/content/drive/MyDrive/Colab Notebooks/Transformer/data/text (1).txt', 'r')
s = sample.read()

In [15]:
# 이스케이프 문자 전처리
f = s.replace('\n', ' ')
data = []

# 문장 파싱(parshing)
for i in sent_tokenize(f) :
  temp = []
  # 문장을 단어로 토큰화하기
  for j in word_tokenize(i) :
    temp.append(j.lower())
  data.append(temp)

In [16]:
# Skip Gram 모델 생성하기
# window: 입력 문장에서 현재 단어와 예측 단어 사이의 거리를 제한함.
# model2 = gensim.models.Word2Vec(data, min_count=1, vector_size=512,
#                                 window=5, sg=1)
model2 = Word2Vec(data, min_count=1, vector_size=512,
                                window=5, sg=1)
print(model2)

Word2Vec<vocab=11770, vector_size=512, alpha=0.025>


In [17]:
# 코싸인 유사도
def similarity(word1, word2) :
  cosine = False  # 기본값
  try :
    a = model2.wv[word1]
    cosine=True
  except KeyError :
    print(word1, ':[unk] key not found in dictionary')
  try :
    b = model2.wv[word2] # a=True is implied
  except KeyError :
    cosine=False # a, b 둘 다 true여야 함.
    print(word2, '[unk] key not not found in dictionary')

  if(cosine==True) :
    b = model2.wv[word2]
    # 코싸인 유사도 계산
    dot = np.dot(a, b)
    norma = np.linalg.norm(a)
    normb = np.linalg.norm(b)
    cos = dot / (norma * normb)

    aa = a.reshape(1, 512)
    ba = b.reshape(1, 512)
    # print('Word1', aa)
    # print('Word2', ba)

    cos_lib = cosine_similarity(aa, ba)
    # print(cos_lib, 'word_similarity')

  if(cosine==False) :
    cos_lib = 0
  return cos_lib

In [18]:
# 사례0 : 데이터셋 및 사전의 단어
word1 = 'freedom'
word2 = 'liberty'
print("Similarity",similarity(word1,word2),word1,word2)

Similarity [[0.357166]] freedom liberty


In [19]:
# 사례1 : 데이터셋이나 사전에 없는 단어
word1 = 'corporations'
word2 = 'right'
print("Similarity",similarity(word1,word2),word1,word2)

corporations :[unk] key not found in dictionary
Similarity 0 corporations right


In [20]:
# 노이즈성 관계
word1 = 'etext'
word2 = 'declaration'
print("Similarity",similarity(word1,word2),word1,word2)

Similarity [[0.5753577]] etext declaration


In [21]:
# 드문 단어
word1 = 'justiciar'
word2 = 'judgement'
print('Similarity', similarity(word1,word2), word1, word2)

Similarity [[0.23686805]] justiciar judgement


In [22]:
# 드문 단어 대체
word1 = 'judge'
word2 = 'judgement'
print('Smilarity', similarity(word1,word2), word1, word2)

Smilarity [[0.19430125]] judge judgement


In [23]:
word1 = 'justiciar'
word2 = 'judge'
print('Smilarity', similarity(word1,word2), word1, word2)

Smilarity [[0.35922092]] justiciar judge


In [24]:
# 수반(함의, entailment)
word1 = 'pay'
word2 = 'debt'
print('Smilarity', similarity(word1,word2), word1, word2)

Smilarity [[0.47841477]] pay debt


### T5 권리장전 샘플

In [28]:
# 모델을 초기화할 때 아케텍처를 표시할지 여부
display_architecture = True

In [29]:
# T5-large 조건부 생성 모델을 가져와
# 텍스트와 T5-large 토크나이저를 생성하기
model = T5ForConditionalGeneration.from_pretrained('t5-large')
tokenizer = T5Tokenizer.from_pretrained('t5-large')
device = torch.device('cpu')

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [30]:
if display_architecture==True :
  print(model.config)

T5Config {
  "_name_or_path": "t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 4096,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "p

In [31]:
if display_architecture==True :
  print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [32]:
if display_architecture==True :
  print(model.encoder)

T5Stack(
  (embed_tokens): Embedding(32128, 1024)
  (block): ModuleList(
    (0): T5Block(
      (layer): ModuleList(
        (0): T5LayerSelfAttention(
          (SelfAttention): T5Attention(
            (q): Linear(in_features=1024, out_features=1024, bias=False)
            (k): Linear(in_features=1024, out_features=1024, bias=False)
            (v): Linear(in_features=1024, out_features=1024, bias=False)
            (o): Linear(in_features=1024, out_features=1024, bias=False)
            (relative_attention_bias): Embedding(32, 16)
          )
          (layer_norm): T5LayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): T5LayerFF(
          (DenseReluDense): T5DenseActDense(
            (wi): Linear(in_features=1024, out_features=4096, bias=False)
            (wo): Linear(in_features=4096, out_features=1024, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
            (act): ReLU()
          )
          (layer_norm): T5LayerNorm()
 

In [33]:
if display_architecture==True :
  print(model.decoder)

T5Stack(
  (embed_tokens): Embedding(32128, 1024)
  (block): ModuleList(
    (0): T5Block(
      (layer): ModuleList(
        (0): T5LayerSelfAttention(
          (SelfAttention): T5Attention(
            (q): Linear(in_features=1024, out_features=1024, bias=False)
            (k): Linear(in_features=1024, out_features=1024, bias=False)
            (v): Linear(in_features=1024, out_features=1024, bias=False)
            (o): Linear(in_features=1024, out_features=1024, bias=False)
            (relative_attention_bias): Embedding(32, 16)
          )
          (layer_norm): T5LayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): T5LayerCrossAttention(
          (EncDecAttention): T5Attention(
            (q): Linear(in_features=1024, out_features=1024, bias=False)
            (k): Linear(in_features=1024, out_features=1024, bias=False)
            (v): Linear(in_features=1024, out_features=1024, bias=False)
            (o): Linear(in_features=1024, out_feat

In [35]:
if display_architecture==True :
  print(model.forward)

<bound method T5ForConditionalGeneration.forward of T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4

In [36]:
def summarize(text, ml) :
  preprocess_text = text.strip().replace('\n','') # \n 제거
  t5_prepared_Text = 'summarize: ' + preprocess_text
  print('Preprocessed and prepared text: \n', t5_prepared_Text)
  tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors='pt').to(device)

  # 요약하기
  summary_ids = model.generate(tokenized_text,
                               num_beams=4,
                               no_repeat_ngram_size=2,
                               min_length=30,
                               max_length=ml,
                               early_stopping=True)

  # decoding
  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return output

In [37]:
# 권리장전 샘플
text ="""
No person shall be held to answer for a capital, or otherwise infamous crime,
unless on a presentment or indictment of a Grand Jury, except in cases arising
in the land or naval forces, or in the Militia, when in actual service
in time of War or public danger; nor shall any person be subject for
the same offense to be twice put in jeopardy of life or limb;
nor shall be compelled in any criminal case to be a witness against himself,
nor be deprived of life, liberty, or property, without due process of law;
nor shall private property be taken for public use without just compensation.
"""
print('Number of characters:', len(text))
summary = summarize(text, 50)
print('\n\nSummarized text: \n', summary)

Number of characters: 590
Preprocessed and prepared text: 
 summarize: No person shall be held to answer for a capital, or otherwise infamous crime,unless on a presentment or indictment of a Grand Jury, except in cases arisingin the land or naval forces, or in the Militia, when in actual servicein time of War or public danger; nor shall any person be subject forthe same offense to be twice put in jeopardy of life or limb;nor shall be compelled in any criminal case to be a witness against himself,nor be deprived of life, liberty, or property, without due process of law;nor shall private property be taken for public use without just compensation.


Summarized text: 
 no person shall be held to answer for a capital, or otherwise infamous crime, unless ona presentment or indictment ofa Grand Jury. nor shall any person be subject for the same offense to be twice put


#### 권리장전 요약2

In [38]:
text ="""
A person must be indicted by a Grand Jury for a capital or infamous crime.
There are excpetions in time of war for a person in the army, navy, or national guard.
A person can not be judged twice for the same offense or put in a situation of double jeopardy of life.
A person can not be asked to be a witness agains herself or himself.
A person cannot be deprived of life, liberty or property without due process of law.
A person must be compensated for property taken for public use.
"""
print('Number of characters:', len(text))
summary = summarize(text, 50)
print('\n\nSummarized text: \n', summary)

Number of characters: 485
Preprocessed and prepared text: 
 summarize: A person must be indicted by a Grand Jury for a capital or infamous crime.There are excpetions in time of war for a person in the army, navy, or national guard.A person can not be judged twice for the same offense or put in a situation of double jeopardy of life.A person can not be asked to be a witness agains herself or himself.A person cannot be deprived of life, liberty or property without due process of law.A person must be compensated for property taken for public use.


Summarized text: 
 a person cannot be deprived of life, liberty or property without due process of law.A person must be compensated for property taken for public use.
